In [1]:
from IPython.core.display import HTML
import ipywidgets as widgets
from IPython.display import display, clear_output
import fsf_input
import fsf_engine
import os 
import fsf_gif
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
import matplotlib.animation as animation

# 一维浅水波方程仿真程序
![问题情景](problem_position.png)

## 1.计算核心

In [2]:
# Declare these variables at the top level to make them globally accessible
Q, H, X_series, T_series = None, None, None, None

def start_calculation(selected_file, chosen_method):
    global Q, H, X_series, T_series  # Declare them as global inside this function
    with result_output:  # Redirect all prints and outputs inside this block to result_output
        clear_output(wait=True)  # Clear the previous output
        
        # The rest of the function remains unchanged
        if not selected_file:
            print("No file selected. Please select a dataset.")
            return

        if not chosen_method:
            print("No calculation method selected. Please choose a method.")
            return

        flag_method = 1 if chosen_method == "matrix_inv" else 2
        filename = f"../inputs/{selected_file}"
        read_data = fsf_input.read(filename)        
        read_data.append(flag_method)

        try:
            [Q, H, X_series, T_series] = fsf_engine.calculation(read_data)
            print(f"Calculation completed for {selected_file} using {chosen_method}.")
            print(f"Shape of Q: {Q.shape}")
        except Exception as e:
            print(f"Error occurred during calculation: {str(e)}")

# Create an output widget with a border for the calculation results
result_output = widgets.Output(layout=widgets.Layout(border='solid 2px black', padding='10px', margin='10px'))

# Create widgets with enhanced styling
button_layout = widgets.Layout(width='250px', height='50px', border='solid 1px black', color='lightblue')

file_combobox = widgets.Dropdown(
    options=[f for f in os.listdir('../inputs/') if os.path.isfile(os.path.join('../inputs/', f))],
    description="Dataset:",
    layout=button_layout
)

method_dropdown = widgets.Dropdown(
    options=["matrix_inv", "double_sweep"],
    description="Method:",
    layout=button_layout
)

start_button = widgets.Button(
    description="Start Calculation",
    layout=button_layout
)

output = widgets.Output()

# Display widgets in a VBox with spacing and alignment to center
# Display widgets in an HBox with increased spacing and alignment to center
main_box = widgets.HBox(
    [file_combobox, method_dropdown, start_button], 
    layout=widgets.Layout(align_items='center', justify_content='center', padding='20px', spacing='50px')  # Increased padding and spacing
)
display(main_box)
display(output)
display(result_output)  # Display the result output below the main box

# Define button actions
def on_button_click(button):
    with output:
        clear_output(wait=True)
        start_calculation(file_combobox.value, method_dropdown.value)

start_button.on_click(on_button_click)

Output()

Output(layout=Layout(border_bottom='solid 2px black', border_left='solid 2px black', border_right='solid 2px b…

# 2.流量动画

In [3]:
def process(N, label, title, Y, X_series, ax):
    ax.set_xlabel('Distance (m)', size=12)
    ax.set_ylabel(label, size=12)
    ax.set_title(title, size=14)
    ims = []
    for n in range(N):
        im, = ax.plot(X_series, Y[10*n], animated=True)
        ims.append([im])
    ani = animation.ArtistAnimation(ax.figure, ims, interval=200, repeat_delay=1000, blit=True)
    return ani

# Define the output widget for discharge with centered alignment
output_widget_discharge = widgets.Output(layout=widgets.Layout(height='300px', align_items='center'))

def generate_discharge_animation(button):
    # Clear the previous animations
    with output_widget_discharge:
        clear_output(wait=True)
        
        fig, ax = plt.subplots(figsize=(8, 2))
        ax.set_xlim(min(X_series), max(X_series))
        ax.set_ylim(min(Q.flatten()), max(Q.flatten()))
        
        N = int(len(T_series)/10)
        ani = process(N, 'Discharge (m^3/s)', 'Discharge changing with distance', Q, X_series, ax)
        display(HTML(ani.to_jshtml()))
        plt.close(fig)
button_layout = widgets.Layout(width='250px', height='50px', border='solid 1px black', color='lightblue')

# Button styling for discharge
generate_discharge_button = widgets.Button(
    description="Generate Discharge Animation", 
    layout=button_layout, 
    style={'button_color': 'lightgray'}
)
generate_discharge_button.on_click(generate_discharge_animation)

# Placeholder empty plot for discharge
with output_widget_discharge:
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.axis('off')
    plt.close(fig)

# Box for discharge button and output
box_discharge = widgets.VBox([generate_discharge_button, output_widget_discharge], layout=widgets.Layout(align_items='center'))

# Surrounding box layout with border
surrounding_layout = widgets.Layout(
    border='solid 2px black', 
    padding='10px', 
    align_items='center',
    width='80%',  # Set this to control the width of the surrounding box
    margin='auto'  # Center the surrounding box
)
# Surrounding box for discharge
surrounding_box_discharge = widgets.VBox([box_discharge], layout=surrounding_layout)
display(surrounding_box_discharge)




# 3.水位动画

In [4]:
# Define the output widget for water level with centered alignment
output_widget_wl = widgets.Output(layout=widgets.Layout(height='300px', align_items='center'))

def generate_water_level_animation(button):
    with output_widget_wl:
        clear_output(wait=True)
        
        fig, ax = plt.subplots(figsize=(8, 2))
        ax.set_xlim(min(X_series), max(X_series))
        ax.set_ylim(min(H.flatten()), max(H.flatten()))
        
        N = int(len(T_series)/10)
        ani = process(N, 'Water level (m)', 'Water level changing with distance', H, X_series, ax)
        display(HTML(ani.to_jshtml()))
        plt.close(fig)

# Button styling for water level
generate_wl_button = widgets.Button(
    description="Generate Water Level Animation", 
    layout=button_layout, 
    style={'button_color': 'lightgray'}
)
generate_wl_button.on_click(generate_water_level_animation)

# Placeholder empty plot for water level
with output_widget_wl:
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.axis('off')
    plt.close(fig)

# Box for water level button and output
box_wl = widgets.VBox([generate_wl_button, output_widget_wl], layout=widgets.Layout(align_items='center'))

# Surrounding box for water level
surrounding_box_wl = widgets.VBox([box_wl], layout=surrounding_layout)
display(surrounding_box_wl)
